In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Have referred the code for data analysis and usage from some notebooks in the competition thread.

In [ ]:
import numpy as np
import json
import pandas as pd

In [ ]:
train_df = pd.read_json(r'/kaggle/input/stanford-covid-vaccine/train.json', lines=True)
if ~train_df.isnull().values.any(): print('No missing values')
train_df.head()


limit = 68
train_df['sequence'] = train_df['sequence'].str.slice(0, limit)
train_df['structure'] = train_df['structure'].str.slice(0, limit)
train_df['predicted_loop_type'] = train_df['predicted_loop_type'].str.slice(0, limit)

test_df = pd.read_json(r'/kaggle/input/stanford-covid-vaccine/test.json', lines=True)
test_df.head()

In [ ]:
sequence_tokens = {'A': 0.2, 'U': 0.4, 'G': 0.6, 'C': 0.8}
structure_tokens = {'(': 0.33, ')': 1, '.': 0.66}
predictedlooptype_tokens = {'S': 0.125, 'M': 0.25, 'I': 0.375, 'E': 0.5, 'X': 0.625, 'H': 0.75, 'B': 0.875}
all_tokens = {'A': 0.2, 'U': 0.4, 'G': 0.6, 'C': 0.8, '(': 0.33, ')': 1, '.': 0.66, 'S': 0.125, 'M': 0.25, 'I': 0.375, 'E': 0.5, 'X': 0.625, 'H': 0.75, 'B': 0.875}

print(all_tokens['A'])
# train_input_cols=['sequence', 'structure', 'predicted_loop_type']
# train_df['tokenized_sequence'] = train_df['tokenized_sequence'].astype(float)
# train_df['tokenized_structure'] = train_df['tokenized_structure'].astype(float)
# train_df['tokenized_predicted_loop_type'] = train_df['tokenized_predicted_loop_type'].astype(float)
pred_cols = ['reactivity', 'deg_Mg_pH10', 'deg_Mg_50C', 'deg_pH10', 'deg_50C']

tokenized_sequence_arr = []
tokenized_structure_arr = []
tokenized_predicted_loop_type_arr = []

for idx in range(0, train_df.shape[0]):
    seq = train_df.iloc[idx, train_df.columns.get_loc('sequence')]
    arr = []
    for char in seq:
        arr.append(float(all_tokens[char]))
    tokenized_sequence_arr.append(arr)
    
    seq = train_df.iloc[idx, train_df.columns.get_loc('structure')]
    arr = []
    for char in seq:
        arr.append(all_tokens[char])
    tokenized_structure_arr.append(arr)

    seq = train_df.iloc[idx, train_df.columns.get_loc('predicted_loop_type')]
    arr = []
    for char in seq:
        arr.append(all_tokens[char])
    tokenized_predicted_loop_type_arr.append(arr)
    
created_df = pd.DataFrame({'tokenized_sequence': tokenized_sequence_arr, 'tokenized_structure': tokenized_structure_arr, 
                           'tokenized_predicted_loop_type': tokenized_predicted_loop_type_arr})
train_input_cols = ['tokenized_sequence', 'tokenized_structure', 'tokenized_predicted_loop_type']
# train_inputs = np.array(
#             created_df[train_input_cols].values.tolist()
#         )
# train_labels = np.array(train_df[pred_cols].values.tolist())
train_inputs= np.transpose(
        np.array(
            created_df[train_input_cols].values.tolist()
        ),
        (0, 2, 1)
    )
train_labels = np.transpose(
        np.array(train_df[pred_cols].values.tolist()),
        (0, 2, 1)
    )
print(train_inputs.shape)
print(train_labels.shape)
# print(train_inputs[0])
# print(train_labels[0])

In [ ]:
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, utils
class MrnaDataset(Dataset):
    """Mrna dataset."""

    def __init__(self, dataframe, labels):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.train_inputs = dataframe
        self.train_labels = labels

    def __len__(self):
        return self.train_inputs.shape[0]

    def __getitem__(self, idx):
        data = self.train_inputs[idx]
        labels = self.train_labels[idx]
        sample = {'inputs': data, 'labels': labels}
#         print(sample)
        return sample

from torch.utils.data.dataloader import default_collate 


def id_collate(batch):
    new_batch = []
    ids = []
    for _batch in batch:
        new_batch.append(_batch['inputs'])
        ids.append(_batch['labels'])
    print(ids)
    return default_collate(new_batch), ids

class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        inputs, labels = sample['inputs'], sample['labels']
        return {'inputs': torch.from_numpy(inputs), 'labels': labels}

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, pred_len):
        super(LSTMModel, self).__init__()
        
        # Number of hidden dimensions
        self.hidden_dim = hidden_dim
        
        # Number of hidden layers
        self.layer_dim = layer_dim
        
        # LSTM
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
        
        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
        self.pred_len = pred_len
    
    def forward(self, x):
        
        # Initialize hidden state with zeros
#         h0 = torch.randn(self.layer_dim, x.shape[0], self.hidden_dim)
#         c0 = torch.randn(self.layer_dim, x.shape[0], self.hidden_dim)
#         h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))
            
        # One time step
#         out, hn = self.lstm(x)
        output, (hn, cn) = self.lstm(x)
        out = output[:, : self.pred_len, :]
#         out = self.fc(out[:, -1, :]) 
        return out, hn

In [ ]:
SAVE_WEIGHTS_PATH = r'/kaggle/input/lstm-weights/weights.pth'
# batch_size, epoch and iteration
batch_size = 64
# n_iters = 8000
# num_epochs = n_iters / (len(features_train) / batch_size)
# num_epochs = int(num_epochs)

# data loader
dataset = MrnaDataset(train_inputs, train_labels)
train_loader = torch.utils.data.DataLoader(dataset, batch_size = batch_size, shuffle = False)
# test_loader = DataLoader(test, batch_size = batch_size, shuffle = False)
    
# Create RNN
input_dim =  3   # input dimension, number of features as input for each lstm cell
hidden_dim = 5  # hidden layer dimension
layer_dim = 107     # number of lstm cells
output_dim = 5
pred_len = 68

# model = melanomaModel().cuda() if params.cuda else melanomaModel()
if torch.cuda.is_available():
    model = LSTMModel(input_dim, hidden_dim, layer_dim, pred_len).cuda()
else:
    model = LSTMModel(input_dim, hidden_dim, layer_dim, pred_len)

# model = LSTMModel(input_dim, hidden_dim, layer_dim, output_dim)

# Cross Entropy Loss 
# error = nn.CrossEntropyLoss()

# SGD Optimizer
learning_rate = 0.001
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
loss_function = nn.MSELoss()
restore_file = True

if restore_file:
    print('restored file')
    checkpoint = torch.load(SAVE_WEIGHTS_PATH)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
# with torch.no_grad():
#     inputs = prepare_sequence(training_data[0][0], word_to_ix)
#     tag_scores = model(inputs)
#     print(tag_scores)

for epoch in range(2):  # again, normally you would NOT do 300 epochs, it is toy data
    print('epoch: ',epoch)
    for index, sampled_batch in enumerate(train_loader):
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        inputs, labels = sampled_batch['inputs'].float(), sampled_batch['labels'].float()
        if torch.cuda.is_available():
            inputs, labels = inputs.cuda(non_blocking=True), labels.cuda(non_blocking=True)
            
#         print(inputs.shape)
#         print(labels.shape)
        model.zero_grad()

        # Step 3. Run our forward pass.
        batch_output, states = model(inputs)
#         print('--------------')
#         print(batch_output.shape)
#         print(states)
#         print(labels.shape)
#         print(batch_output)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(batch_output, labels)
#         print(loss)
        loss.backward()
        optimizer.step()
    print(loss)
#     torch.save({
#         'epoch': epoch+1,
#         'model_state_dict': model.state_dict(),
#         'optimizer_state_dict': optimizer.state_dict(),
# #             'loss': loss
# #             is_best=is_best
#         }, SAVE_WEIGHTS_PATH)

# See what the scores are after training
# with torch.no_grad():
#     tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
#     print(tag_scores)

In [ ]:
test_public_df = test_df.query("seq_length == 107")
test_private_df = test_df.query("seq_length == 130")
limit = 68
test_public_df['sequence'] = test_public_df['sequence'].str.slice(0, limit)
test_public_df['structure'] = test_public_df['structure'].str.slice(0, limit)
test_public_df['predicted_loop_type'] = test_public_df['predicted_loop_type'].str.slice(0, limit)

limit = 91
test_private_df['sequence'] = test_private_df['sequence'].str.slice(0, limit)
test_private_df['structure'] = test_private_df['structure'].str.slice(0, limit)
test_private_df['predicted_loop_type'] = test_private_df['predicted_loop_type'].str.slice(0, limit)


In [ ]:
pred_cols = ['reactivity', 'deg_Mg_pH10', 'deg_Mg_50C', 'deg_pH10', 'deg_50C']

def tokenize_inputs(df):
    tokenized_sequence_arr = []
    tokenized_structure_arr = []
    tokenized_predicted_loop_type_arr = []
    ids = []
    for idx in range(0, df.shape[0]):
        ids.append(df.iloc[idx, df.columns.get_loc('id')])
        seq = df.iloc[idx, df.columns.get_loc('sequence')]
        arr = []
        for char in seq:
            arr.append(float(all_tokens[char]))
        tokenized_sequence_arr.append(arr)

        seq = df.iloc[idx, df.columns.get_loc('structure')]
        arr = []
        for char in seq:
            arr.append(all_tokens[char])
        tokenized_structure_arr.append(arr)

        seq = df.iloc[idx, df.columns.get_loc('predicted_loop_type')]
        arr = []
        for char in seq:
            arr.append(all_tokens[char])
        tokenized_predicted_loop_type_arr.append(arr)

    created_df = pd.DataFrame({'tokenized_sequence': tokenized_sequence_arr, 'tokenized_structure': tokenized_structure_arr, 
                           'tokenized_predicted_loop_type': tokenized_predicted_loop_type_arr, 'id': ids})
    created_ids = pd.DataFrame({'id': ids})
    return created_df, created_ids

test_input_cols = ['tokenized_sequence', 'tokenized_structure', 'tokenized_predicted_loop_type']
train_input_cols = ['tokenized_sequence', 'tokenized_structure', 'tokenized_predicted_loop_type']

created_df_public, created_ids_public = tokenize_inputs(test_public_df)
created_df_private, created_ids_private = tokenize_inputs(test_private_df)

test_inputs_public = np.transpose(
        np.array(
            created_df_public[train_input_cols].values.tolist()
        ),
        (0, 2, 1)
    )
test_ids_public = np.transpose(
        np.array(
            created_ids_public[['id']].values.tolist()
        ),
        (0, 1)
    )
test_inputs_private = np.transpose(
        np.array(
            created_df_private[train_input_cols].values.tolist()
        ),
        (0, 2, 1)
    )
test_ids_private = np.transpose(
        np.array(
            created_ids_private[['id']].values.tolist()
        ),
        (0, 1)
    )
print(test_inputs_public.shape)
print(test_ids_public.shape)
print(test_inputs_private.shape)
print(test_ids_private.shape)

In [ ]:
    
# Create RNN
input_dim =  3   # input dimension, number of features as input for each lstm cell
hidden_dim = 5  # hidden layer dimension
layer_dim = 107     # number of lstm cells
output_dim = 5
pred_len = 68

# pred_cols = ['reactivity', 'deg_Mg_pH10', 'deg_Mg_50C', 'deg_pH10', 'deg_50C']
# model = melanomaModel().cuda() if params.cuda else melanomaModel()
if torch.cuda.is_available():
    model = LSTMModel(input_dim, hidden_dim, layer_dim, pred_len).cuda()
else:
    model = LSTMModel(input_dim, hidden_dim, layer_dim, pred_len)

checkpoint = torch.load(SAVE_WEIGHTS_PATH)
model.load_state_dict(checkpoint['model_state_dict'])
values = []

inputs, labels = test_inputs_public, test_ids_public
if torch.cuda.is_available():
    inputs = torch.from_numpy(inputs).float().cuda(non_blocking=True)
batch_output, states = model(inputs)
outputs = batch_output.data.cpu().numpy()
outputs = np.array(np.transpose(outputs, (0, 2, 1)))
#     print(reactivity)
print(labels[0])
reactivity = np.array(outputs[:, 0, :]).tolist()
deg_Mg_pH10 = np.array(outputs[:, 0, :]).tolist()
deg_Mg_50C = np.array(outputs[:, 0, :]).tolist()
deg_pH10 = np.array(outputs[:, 0, :]).tolist()
deg_50C = np.array(outputs[:, 0, :]).tolist()
for i in range(outputs.shape[0]):
    for j in range(68):
        values.append([str(labels[i][0])+ '_' + str(j), reactivity[i][j], deg_Mg_pH10[i][j], deg_Mg_50C[i][j], deg_pH10[i][j], deg_50C[i][j]])



In [ ]:
# Create RNN
input_dim =  3   # input dimension, number of features as input for each lstm cell
hidden_dim = 5  # hidden layer dimension
layer_dim = 107    # number of lstm cells
output_dim = 5
pred_len = 91

# pred_cols = ['reactivity', 'deg_Mg_pH10', 'deg_Mg_50C', 'deg_pH10', 'deg_50C']
# model = melanomaModel().cuda() if params.cuda else melanomaModel()
if torch.cuda.is_available():
    model = LSTMModel(input_dim, hidden_dim, layer_dim, pred_len).cuda()
else:
    model = LSTMModel(input_dim, hidden_dim, layer_dim, pred_len)

checkpoint = torch.load(SAVE_WEIGHTS_PATH)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

inputs, labels = test_inputs_private, test_ids_private
if torch.cuda.is_available():
    inputs = torch.from_numpy(inputs).float().cuda(non_blocking=True)
batch_output, states = model(inputs)
outputs = batch_output.data.cpu().numpy()
outputs = np.array(np.transpose(outputs, (0, 2, 1)))
#     print(reactivity)
print(labels[0])
reactivity = np.array(outputs[:, 0, :]).tolist()
deg_Mg_pH10 = np.array(outputs[:, 0, :]).tolist()
deg_Mg_50C = np.array(outputs[:, 0, :]).tolist()
deg_pH10 = np.array(outputs[:, 0, :]).tolist()
deg_50C = np.array(outputs[:, 0, :]).tolist()
for i in range(outputs.shape[0]):
    for j in range(91):
        values.append([str(labels[i][0])+ '_' + str(j), reactivity[i][j], deg_Mg_pH10[i][j], deg_Mg_50C[i][j], deg_pH10[i][j], deg_50C[i][j]])


    
sample_submission = pd.DataFrame(values, columns=['id_seqpos', 'reactivity', 'deg_Mg_pH10', 'deg_Mg_50C', 'deg_pH10', 'deg_50C'])
sample_submission.to_csv('submission.csv', index=False)